In [206]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rcParams
import platform

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [207]:
### matplotlib 에서 한글 및 음수 표현이 깨지는 현상 수정

system = platform.system()

if system == "Windows":     # Windows
    rcParams['font.family'] = 'Malgun Gothic'
elif system == "Darwin":    # macOS
    rcParams['font.family'] = 'AppleGothic'

rcParams['axes.unicode_minus'] = False

In [ ]:
### 데이터 체크 클래스 선언

class DataCheck() :
    
    # 생성 시 기존 데이터를 넣어둠
    def __init__(self, df) :
        self.raw_df = df
    
    # continuous한 컬럼과 categorical한 컬럼을 반환
    @staticmethod
    def return_cols(df, type, boundary=15) :    # type : ('continuous', 'categorical'), boundary : 두 분류를 결정할 서로 다른 요소의 수
        cols = []
        if type == 'continuous' :
            for col in df.columns:
                if df[col].nunique(dropna=True) >= boundary :
                    cols.append(col)
        elif type == 'categorical' :
            for col in df.columns:
                if df[col].nunique(dropna=True) < boundary :
                    cols.append(col)
        else :
            ValueError('Wrong type.')

        return cols
    
    # 정보 출력
    def print_info(self, df = None) :
        if df is None :         # 데이터를 전달하지 않으면 기존 데이터를 활용
            df = self.raw_df

        print('------ Data Info -----')
        df.info()
        print('\n----- Data Describe -----')
        display(df.describe())

    # 각 컬럼들의 값 출력
    def print_value_counts(self, df = None) :
        if df is None :         # 데이터를 전달하지 않으면 기존 데이터를 활용
            df = self.raw_df
        
        for col in df.columns:
            if df[col].nunique(dropna=True) > 15 :  # 각기 다른 값이 15개 이상인 Continuous 한 값들은 출력하지 않음
                print(col)
                print('continuous')
                print("-"*20)
                continue

            value_counts = df[col].fillna("NA").value_counts(dropna=False)  # 결측치는 NA로 처리 후 출력
            print(value_counts)
            print("-" * 20)

    # 데이터 to csv
    def save_to_csv(self, df):
        df.to_csv('Tree_Data_processing.csv', index=False)
    

In [ ]:
### 데이터 정제 클래스 선언

class DataModify () :
    
    # SMOTE 사용한 데이터 오버 샘플링
    @staticmethod
    def data_oversampling(df) :
        df_oversampled = df
        return df_oversampled
    
    # 이상치를 제거
    @staticmethod
    def data_anomaly_del(df, return_anomaly=True) :
        df_cleaned = df
        df_anomaly = None
        return df_cleaned, df_anomaly
        

In [ ]:
### 데이터 전처리 클래스 선언

class DataPreprocessing () :

    def __init__(self, df) :
        self.raw_df = df        # 수정 전 기본 데이터
        self.df = df            # 수정할 데이터
        self.categories = {}    # 인코딩한 카테고리
        self.drop_cols = []

    # 드랍할 컬럼을 설정
    def set_drop_cols(self, cols):
        self.drop_cols = cols

    # 수정 전의 데이터를 불러옴
    def call_raw_data(self) :
        return self.raw_df
    
    # 수정된 상태의 데이터를 불러옴, 데이터를 나누지 않은 상태로 불러옴
    def call_full_data(self) :
        return self.df
    
    # 데이터 드랍
    def drop_data(self, columns=None) :
        self.df = self.df.drop(columns=columns)
    
    # 결측치 처리
    def fill_na(self) :
        self.df['Harvest'] = self.df['Harvest'].fillna('O') # Harvest는 Binary 한 값
        self.df['Alive'] = self.df['Alive'].fillna('O')     # Alive는 Binary 한 값
        self.df['EMF'] = self.df['EMF'].fillna(0)           # EMF는 균의 비율로, 측정되지 않은 데이터는 균이 없는 것으로 추정

    # categorical한 데이터 encoding
    def category_encoding(self, encoding='label') :
        # encoding : 'label' - 라벨 인코딩, 'onehot' - One-hot 인코딩
        # categories = {'encoding_type':..., '컬럼명':{'요소':'라벨', '요소':'라벨', ...}, ...}
        # categorical한 데이터 = 데이터가 가지는 서로 다른 값이 15개 미만

        categorical_col = DataCheck.return_cols(self.df, 'categorical')       # Categorical한 column의 컬럼명을 선택

        df_encoded = self.df.copy() # 인코딩할 데이터

        if encoding == 'label' : # 라벨 형식으로 인코딩
            self.categories['encoding_type'] = 'label'

            for col in categorical_col:
                unique_vals = df_encoded[col].unique()
                label = {val: i for i, val in enumerate(unique_vals)}
                df_encoded[col] = df_encoded[col].map(label)
                self.categories[col] = label

        elif encoding == 'onehot' : # One-hot 인코딩
            self.categories['encoding_type'] = 'onehot'
            
            for col in categorical_col:
                dummies = pd.get_dummies(df_encoded[col], prefix=col)
                df_encoded = pd.concat([df_encoded.drop(columns=[col]), dummies], axis=1)
                self.categories[col] = dummies.columns.tolist()

        else:   # 이상한 값이 들어오면 Value Error 발생
            raise ValueError(f"알 수 없는 encoding_type: {encoding}")

        return df_encoded   # 인코딩된 데이터를 반환
    
    # encoding된 데이터 decoding
    def category_decoding(self, df = None, categories=None) :
        if df is None :                 # 넣어준 인자가 없다면 기존 인스턴스에 저장된 데이터를 디코딩 수행
            df_decoded = self.df.copy()
        else:                           # 넣어준 인자가 있다면 넣어준 데이터에 대해 디코딩 수행
            df_decoded = df.copy()

        if categories is not None :
            self.categories = categories

        encoding_type = self.categories.get('encoding_type', None)  # categories 데이터에 저장된 인코딩 타입에 따라 인코딩 타입 설정

        if encoding_type == 'label':    # 라벨 인코딩일 경우
            for col, mapping in self.categories.items():
                if col == 'encoding_type':
                    continue
                # {원본값: 숫자} → {숫자: 원본값}
                reverse_map = {v: k for k, v in mapping.items()}
                df_decoded[col] = df_decoded[col].map(reverse_map)

        elif encoding_type == 'onehot': # One-hot 인코딩일 경우
            for col, dummy_cols in self.categories.items():
                if col == 'encoding_type':
                    continue
                # 각 mapping column 에서 값이 True인 행을 찾아 역으로 mapping
                def decode_row(row):
                    for dummy_col in dummy_cols:
                        if row[dummy_col] == 1:
                            return dummy_col.replace(f"{col}_", "")
                    return None
                
                df_decoded[col] = df_decoded.apply(decode_row, axis=1)
                df_decoded = df_decoded.drop(columns=dummy_cols)

        else:
            raise ValueError(f"알 수 없는 encoding_type: {encoding_type}")

        return df_decoded           # dictionary 형태 categories 를 받아서, 해당 데이터를 기반으로 디코딩 후, 테이블 반환
    
    # 날짜 기재 방식 통일
    def set_date(self):
        def date_modify(x):
            x = str(x)
            if '/' in x:
                return x.split('/')[0]
            else:
                return x 
        self.df['PlantDate'] = self.df['PlantDate'].apply(date_modify)
    
    # 생존, 실험 중단(수확), 사망 이벤트 -> label data로 병합
    def merge_label(self) :
        # Event가 NULL인 데이터는 pred_data로 따로 저장
        self.df.loc[self.df["Event"] == 1, "Alive"] = 0
        self.df.loc[self.df["Harvest"] == 'X', "Alive"] = 1
        self.df.loc[self.df["Alive"] == 'X', "Alive"] = 2

        self.df.drop(columns=['Event','Harvest'], inplace=True)

        return    # Event, Harvest, Alive를 병합하여 Dead, Harvest, Alive 3개의 카테고리로 분류
    
    # 데이터와 라벨 분리, 학습 데이터와 테스트 데이터 분리
    def split_feature_label(self) :
        X = self.df.drop(columns='Alive')
        y = self.df['Alive']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

        return [X_train, y_train], [X_test, y_test]     # train set과 test set을 반환
    
    # feature 데이터 스케일링
    def scale_data(self, train_set, test_set) :

        X_train = train_set[0]
        X_test = test_set[0]

        continuous_cols = DataCheck.return_cols(self.df, 'continuous')
        
        scaler = StandardScaler()

        X_train_scaled = X_train.copy()
        X_train_scaled[continuous_cols] = scaler.fit_transform(X_train[continuous_cols])
        train_set_scaled = [X_train_scaled, train_set[1]]

        X_test_scaled = X_test.copy()
        X_test_scaled[continuous_cols] = scaler.transform(X_test[continuous_cols])
        test_set_scaled = [X_test_scaled, test_set[1]]

        return train_set_scaled, test_set_scaled   # 값의 종류가 15개가 넘는, continuous 하고 숫자를 가지는 값들만 scaled
        
    # 전처리 과정 일괄 수행 (encoding, split은 선택)
    def run(self, encoding=None, split=False) :

        self.drop_data(self.drop_cols)
        
        self.fill_na()
        self.set_date()
        self.merge_label()

        if encoding is not None :       # 데이터를 인코딩
            self.df = self.category_encoding(encoding)

        if split is False :         # 데이터를 나누지 않는다면 데이터를 그대로 반환
            return self.df
        
        train_set, test_set = self.split_feature_label()
        train_set, test_set = self.scale_data(train_set, test_set)
        return train_set, test_set

In [211]:
### 데이터 시각화 클래스 선언

class DataVisualize () :

    # 생성 시 기존 데이터를 넣어둠
    def __init__(self, df) :
        self.raw_df = df

In [212]:
### 데이터 로드

input_file_path = './data/Tree_data.csv'

df = pd.read_csv(input_file_path)

### 인스턴스 생성

dc = DataCheck(df)
dp = DataPreprocessing(df)
dv = DataVisualize(df)

In [213]:
# 정보 출력

dc.print_info(df)

------ Data Info -----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2783 entries, 0 to 2782
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   No           2783 non-null   int64  
 1   Plot         2783 non-null   int64  
 2   Subplot      2783 non-null   object 
 3   Species      2783 non-null   object 
 4   Light_ISF    2783 non-null   float64
 5   Light_Cat    2783 non-null   object 
 6   Core         2783 non-null   int64  
 7   Soil         2783 non-null   object 
 8   Adult        2783 non-null   object 
 9   Sterile      2783 non-null   object 
 10  Conspecific  2783 non-null   object 
 11  Myco         2783 non-null   object 
 12  SoilMyco     2783 non-null   object 
 13  PlantDate    2783 non-null   object 
 14  AMF          2783 non-null   float64
 15  EMF          1283 non-null   float64
 16  Phenolics    2783 non-null   float64
 17  Lignin       2783 non-null   float64
 18  NSC          2783 non-nul

,No,Plot,Light_ISF,Core,AMF,EMF,Phenolics,Lignin,NSC,Census,Time,Event
count,2783.000000,2783.000000,2783.000000,2783.000000,2783.000000,1283.00000,2783.000000,2783.000000,2783.000000,2783.000000,2783.000000,2782.000000
mean,3914.513834,9.561624,0.085707,2016.648940,20.553069,26.47675,1.933105,15.759792,14.219641,15.282070,53.487244,0.570453
std,2253.515063,5.203659,0.025638,0.477387,12.309587,16.63689,1.969842,6.779607,4.298271,9.166555,32.082942,0.495100
min,3.000000,1.000000,0.032000,2016.000000,0.000000,0.00000,-1.350000,2.230000,4.300000,4.000000,14.000000,0.000000
25%,1971.000000,5.000000,0.066000,2016.000000,13.400000,13.78000,0.170000,10.355000,11.605000,7.000000,24.500000,0.000000
50%,3932.000000,10.000000,0.082000,2017.000000,18.000000,27.72000,0.750000,14.040000,12.660000,13.000000,45.500000,1.000000
75%,5879.000000,14.000000,0.100000,2017.000000,24.445000,35.71000,3.780000,21.115000,17.275000,18.000000,63.000000,1.000000
max,7772.000000,18.000000,0.161000,2017.000000,100.000000,87.50000,6.100000,32.770000,29.450000,33.000000,115.500000,1.000000


In [214]:
# 컬럼값 출력

dc.print_value_counts(df)

No
continuous
--------------------
Plot
continuous
--------------------
Subplot
A    701
D    666
B    663
C    646
E    107
Name: count, dtype: int64
--------------------
Species
Acer saccharum     751
Prunus serotina    749
Quercus alba       673
Quercus rubra      610
Name: count, dtype: int64
--------------------
Light_ISF
continuous
--------------------
Light_Cat
Med     1474
Low     1005
High     304
Name: count, dtype: int64
--------------------
Core
2017    1806
2016     977
Name: count, dtype: int64
--------------------
Soil
Sterile                  423
Prunus serotina          413
Quercus rubra            402
Acer saccharum           397
Populus grandidentata    391
Quercus alba             381
Acer rubrum              376
Name: count, dtype: int64
--------------------
Adult
continuous
--------------------
Sterile
Non-Sterile    2360
Sterile         423
Name: count, dtype: int64
--------------------
Conspecific
Heterospecific    1974
Sterilized         423
Conspecific        

In [215]:
# 이상치 확인

print(df[df['Event'].isna()])

        No  Plot Subplot       Species  ...   Time Event  Harvest Alive
2781  7771    18       D  Quercus alba  ...  115.5   NaN      NaN   NaN

[1 rows x 24 columns]


In [216]:
### 드랍할 컬럼 설정
drop_cols = ['No', 'Plot','Light_Cat', 'Adult', 'Subplot', 'Core', 'Census']
dp.set_drop_cols(drop_cols)

encode = dp.run(encoding='onehot')
display(encode)

,Light_ISF,AMF,EMF,Phenolics,Lignin,NSC,Time,Species_Acer saccharum,Species_Prunus serotina,Species_Quercus alba,Species_Quercus rubra,Soil_Acer rubrum,Soil_Acer saccharum,Soil_Populus grandidentata,Soil_Prunus serotina,Soil_Quercus alba,Soil_Quercus rubra,Soil_Sterile,Sterile_Non-Sterile,Sterile_Sterile,Conspecific_Conspecific,Conspecific_Heterospecific,Conspecific_Sterilized,Myco_AMF,Myco_EMF,SoilMyco_AMF,SoilMyco_EMF,SoilMyco_Sterile,PlantDate_5,PlantDate_6,Alive_0,Alive_1,Alive_2,Alive_O
0,0.106,22.00,0.00,-0.56,13.86,12.15,14.0,True,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,False,False,True,True,False,False,False
1,0.106,15.82,31.07,5.19,20.52,19.29,115.5,False,False,True,False,False,False,False,False,False,True,False,True,False,False,True,False,False,True,False,True,False,True,False,False,False,True,False
2,0.106,24.45,28.19,3.36,24.74,15.01,63.0,False,False,False,True,False,False,False,True,False,False,False,True,False,False,True,False,False,True,True,False,False,True,False,True,False,False,False
3,0.080,22.23,0.00,-0.71,14.29,12.36,14.0,True,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,False,False,True,True,False,False,False
4,0.060,21.15,0.00,-0.58,10.85,11.20,14.0,True,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2778,0.111,40.89,0.00,0.83,9.15,11.88,56.0,False,True,False,False,False,False,True,False,False,False,False,True,False,False,True,False,True,False,False,True,False,False,True,True,False,False,False
2779,0.118,15.47,32.82,4.88,19.01,23.50,56.0,False,False,True,False,True,False,False,False,False,False,False,True,False,False,True,False,False,True,True,False,False,True,False,True,False,False,False
2780,0.118,11.96,37.67,5.51,21.13,19.10,56.0,False,False,True,False,False,False,False,False,False,True,False,True,False,False,True,False,False,True,False,True,False,True,False,True,False,False,False
2781,0.161,16.99,22.51,4.28,19.38,21.36,115.5,False,False,True,False,False,False,False,False,False,False,True,False,True,False,False,True,False,True,False,False,True,True,False,False,False,False,True


In [217]:
display(dp.category_decoding(encode))

,Light_ISF,AMF,EMF,Phenolics,Lignin,NSC,Time,Species,Soil,Sterile,Conspecific,Myco,SoilMyco,PlantDate,Alive
0,0.106,22.00,0.00,-0.56,13.86,12.15,14.0,Acer saccharum,Prunus serotina,Non-Sterile,Heterospecific,AMF,AMF,6,0
1,0.106,15.82,31.07,5.19,20.52,19.29,115.5,Quercus alba,Quercus rubra,Non-Sterile,Heterospecific,EMF,EMF,5,2
2,0.106,24.45,28.19,3.36,24.74,15.01,63.0,Quercus rubra,Prunus serotina,Non-Sterile,Heterospecific,EMF,AMF,5,0
3,0.080,22.23,0.00,-0.71,14.29,12.36,14.0,Acer saccharum,Prunus serotina,Non-Sterile,Heterospecific,AMF,AMF,6,0
4,0.060,21.15,0.00,-0.58,10.85,11.20,14.0,Acer saccharum,Prunus serotina,Non-Sterile,Heterospecific,AMF,AMF,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2778,0.111,40.89,0.00,0.83,9.15,11.88,56.0,Prunus serotina,Populus grandidentata,Non-Sterile,Heterospecific,AMF,EMF,6,0
2779,0.118,15.47,32.82,4.88,19.01,23.50,56.0,Quercus alba,Acer rubrum,Non-Sterile,Heterospecific,EMF,AMF,5,0
2780,0.118,11.96,37.67,5.51,21.13,19.10,56.0,Quercus alba,Quercus rubra,Non-Sterile,Heterospecific,EMF,EMF,5,0
2781,0.161,16.99,22.51,4.28,19.38,21.36,115.5,Quercus alba,Sterile,Sterile,Sterilized,EMF,Sterile,5,O


In [ ]:
# dc.save_to_csv(encode)